In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
from ikrlib import *


if not os.path.isfile("SUR_projekt2022-2023.zip"):
    !wget https://www.fit.vutbr.cz/study/courses/SUR/public/projekt_2022-2023/SUR_projekt2022-2023.zip
    !unzip SUR_projekt2022-2023.zip


In [3]:
CLASSES = 31

train_x = np.empty((0,80,80))
train_y = np.empty((0),dtype=int)

# TODO mozna bych zkonkatenoval mfcc do train_x
for i in range(1,CLASSES+1): 
    train_i = np.array(list(png2fea('train/'+str(i)).values()))
    label_i = np.full(len(train_i),i)
    #add to train_x
    train_x = np.concatenate((train_x, train_i), axis=0)
    train_y = np.concatenate((train_y,label_i),axis=0)


print(train_x.shape, train_y.shape)
sample1 = train_x[0]
print(train_x.shape)

Processing file:  train/1/f401_01_f18_i0_0.png
Processing file:  train/1/f401_01_r09_i0_0.png
Processing file:  train/1/f401_02_f18_i0_0.png
Processing file:  train/1/f401_02_r09_i0_0.png
Processing file:  train/1/f401_03_f18_i0_0.png
Processing file:  train/1/f401_03_r09_i0_0.png
Processing file:  train/2/f402_01_f18_i0_0.png
Processing file:  train/2/f402_01_r09_i0_0.png
Processing file:  train/2/f402_02_f18_i0_0.png
Processing file:  train/2/f402_02_r09_i0_0.png
Processing file:  train/2/f402_03_f18_i0_0.png
Processing file:  train/2/f402_03_r09_i0_0.png
Processing file:  train/3/f403_01_f18_i0_0.png
Processing file:  train/3/f403_01_r09_i0_0.png
Processing file:  train/3/f403_02_f18_i0_0.png
Processing file:  train/3/f403_02_r09_i0_0.png
Processing file:  train/3/f403_03_f18_i0_0.png
Processing file:  train/3/f403_03_r09_i0_0.png
Processing file:  train/4/f404_01_f18_i0_0.png
Processing file:  train/4/f404_01_r09_i0_0.png
Processing file:  train/4/f404_02_f18_i0_0.png
Processing fi

In [4]:
flat = train_x.reshape(train_x.shape[0],-1)
print(flat.shape)


(186, 6400)


In [5]:
#PCA 
cov = np.cov(flat.T)

eig_vals, eig_vecs = np.linalg.eig(cov)

# sort
eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]
eig_pairs.sort(key=lambda x: x[0], reverse=True)

# print first 10
for i in range(10):
    print(eig_pairs[i])



(3495891.46917172, array([0.01172124+0.j, 0.01166567+0.j, 0.01184372+0.j, ...,
       0.00414626+0.j, 0.00372054+0.j, 0.00408837+0.j]))
(1686240.5868637068, array([ 0.00631871+0.j,  0.00637294+0.j,  0.00634687+0.j, ...,
       -0.01279252+0.j, -0.01188726+0.j, -0.01117919+0.j]))
(1277218.503911742, array([ 0.00826392+0.j,  0.00794732+0.j,  0.00760031+0.j, ...,
       -0.0305899 +0.j, -0.0312002 +0.j, -0.03087405+0.j]))
(753107.534288279, array([0.00242878+0.j, 0.00178403+0.j, 0.00142484+0.j, ...,
       0.01180617+0.j, 0.01322847+0.j, 0.01302666+0.j]))
(600944.5773356641, array([ 0.01549813+0.j,  0.01630812+0.j,  0.01732935+0.j, ...,
       -0.01664399+0.j, -0.01927929+0.j, -0.0207143 +0.j]))
(442296.06038017274, array([0.0007492 +0.j, 0.00115167+0.j, 0.00331759+0.j, ...,
       0.02805571+0.j, 0.02802446+0.j, 0.02808201+0.j]))
(337629.6794181109, array([-0.01639027+0.j, -0.01490219+0.j, -0.01564632+0.j, ...,
        0.00204503+0.j, -0.00035607+0.j, -0.00132261+0.j]))
(303297.031178770

In [13]:
# convert to real with real_if_close
eig_pairs = [(np.real_if_close(eig_pairs[i][0]), np.real(eig_pairs[i][1])) for i in range(len(eig_pairs))]

# print first 10 vectors
for i in range(20):
    print(eig_pairs[i][1])

NUM_VECTORS = 100
# transform to first $NUM_VECTORS eigenvectors
train_x_pca = np.array([np.dot(eig_pairs[i][1], flat.T) for i in range(NUM_VECTORS)]).T
print(train_x_pca.shape,len(eig_pairs))

[0.01172124 0.01166567 0.01184372 ... 0.00414626 0.00372054 0.00408837]
[ 0.00631871  0.00637294  0.00634687 ... -0.01279252 -0.01188726
 -0.01117919]
[ 0.00826392  0.00794732  0.00760031 ... -0.0305899  -0.0312002
 -0.03087405]
[0.00242878 0.00178403 0.00142484 ... 0.01180617 0.01322847 0.01302666]
[ 0.01549813  0.01630812  0.01732935 ... -0.01664399 -0.01927929
 -0.0207143 ]
[0.0007492  0.00115167 0.00331759 ... 0.02805571 0.02802446 0.02808201]
[-0.01639027 -0.01490219 -0.01564632 ...  0.00204503 -0.00035607
 -0.00132261]
[ 0.00602481  0.00544649  0.00363165 ... -0.02207884 -0.01965973
 -0.01882933]
[-0.00313349 -0.00503539 -0.00600801 ...  0.00961892  0.01336429
  0.01600403]
[ 0.00574325  0.00664709  0.00887673 ... -0.02212585 -0.02359261
 -0.02196443]
[0.02696258 0.02889748 0.02777015 ... 0.04916014 0.05415163 0.05405731]
[-0.01703368 -0.0197114  -0.02214867 ...  0.04535104  0.04503892
  0.04218379]
[ 0.00814235  0.00659399  0.00827429 ... -0.00305738 -0.00735812
 -0.00875427]
[ 

In [15]:
#train logistic regression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

model = LogisticRegression().fit(train_x_pca, train_y)
y_pred = model.predict(train_x_pca)
print(accuracy_score(y_test, y_pred))



/home/lada/repo/SUR/venv/lib64/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


NameError: name 'y_test' is not defined

In [8]:
#train data accuarcy
predict_y = gmm.predict(train_x_pca) + 1
print(predict_y)
print(train_y)

print("Train data accuarcy: ", np.sum(predict_y == train_y)/len(train_y))

[ 7 12 29 29 12 12  7  7  7  7  7  7 11 11  6 10  2 11  2  2  2  2 26 26
  9  9  9  9  4  2  8  8 20 19 26 26 20 20 24  3 30 30 20 20 19 19 19 19
 22 22 22 22  8  8 18 18  5  6  5  5 17 17 11 11 11 11 31 31 27 27 23 23
 13 12 13 13 24 24  4  4 20 25  4  4  2 25  2  2  2  2 16  6 16 16 16 16
 18 14 10 10 14 14 21 21 25 25 14 14  5 15  1  1 14 14 28 26 15 15 15 15
 19 25  5  5  6  6 25 25 25 11 26 26  6 11 11 11  6 18 14 18 28 28 14 14
  6 18 20 25 19 19 31 25 20 14 14 14  5 25 10 10 10 10 18 18 18  5 28 28
 14 14  6  6  1  1 18 18 25 25  2 14 18 10 10 10 19 19]
[ 1  1  1  1  1  1  2  2  2  2  2  2  3  3  3  3  3  3  4  4  4  4  4  4
  5  5  5  5  5  5  6  6  6  6  6  6  7  7  7  7  7  7  8  8  8  8  8  8
  9  9  9  9  9  9 10 10 10 10 10 10 11 11 11 11 11 11 12 12 12 12 12 12
 13 13 13 13 13 13 14 14 14 14 14 14 15 15 15 15 15 15 16 16 16 16 16 16
 17 17 17 17 17 17 18 18 18 18 18 18 19 19 19 19 19 19 20 20 20 20 20 20
 21 21 21 21 21 21 22 22 22 22 22 22 23 23 23 23 23 23 24 24 24 24 2

In [9]:
#create Test data
test_x = np.empty((0,80,80))
test_y = np.empty((0),dtype=int)

for i in range(1,CLASSES+1): 
    test_i = np.array(list(png2fea('dev/'+str(i)).values()))
    label_i = np.full(len(test_i),i)
    #add to test_x
    test_x = np.concatenate((test_x, test_i), axis=0)
    test_y = np.concatenate((test_y,label_i),axis=0)
print(test_x.shape, test_y.shape)

# flatten test data
test_x = test_x.reshape(test_x.shape[0],-1)

# transform to first 50 eigenvectors
test_x_pca = np.array([np.dot(eig_pairs[i][1], test_x.T) for i in range(NUM_VECTORS)]).T
print(test_x_pca.shape)


Processing file:  dev/1/f401_04_f18_i0_0.png
Processing file:  dev/1/f401_04_r09_i0_0.png
Processing file:  dev/2/f402_04_f18_i0_0.png
Processing file:  dev/2/f402_04_r09_i0_0.png
Processing file:  dev/3/f403_04_f18_i0_0.png
Processing file:  dev/3/f403_04_r09_i0_0.png
Processing file:  dev/4/f404_04_f18_i0_0.png
Processing file:  dev/4/f404_04_r09_i0_0.png
Processing file:  dev/5/f405_04_f18_i0_0.png
Processing file:  dev/5/f405_04_r09_i0_0.png
Processing file:  dev/6/f406_04_f18_i0_0.png
Processing file:  dev/6/f406_04_r09_i0_0.png
Processing file:  dev/7/f407_04_f18_i0_0.png
Processing file:  dev/7/f407_04_r09_i0_0.png
Processing file:  dev/8/f408_04_f18_i0_0.png
Processing file:  dev/8/f408_04_r09_i0_0.png
Processing file:  dev/9/f409_04_f18_i0_0.png
Processing file:  dev/9/f409_04_r09_i0_0.png
Processing file:  dev/10/f410_04_f18_i0_0.png
Processing file:  dev/10/f410_04_r09_i0_0.png
Processing file:  dev/11/f411_04_f18_i0_0.png
Processing file:  dev/11/f411_04_r09_i0_0.png
Proces

In [10]:
#test data accuarcy
predict_y = gmm.predict(test_x_pca) + 1
print(predict_y)
print(test_y)
print("Test data accuarcy: ", np.sum(predict_y == test_y)/len(test_y))


[13 26  9  9 26 14 25 25  2 27 26 26 26 26 26 26  8  8 26 26 26 26 11 11
 14 14  4  4  2 14 12  7 26 15 14 14 26 14 26 26  6  6 26 26 26 26 14 14
 26 26 26 26 26 14 14 14 14 14 26 26 26 14]
[ 1  1  2  2  3  3  4  4  5  5  6  6  7  7  8  8  9  9 10 10 11 11 12 12
 13 13 14 14 15 15 16 16 17 17 18 18 19 19 20 20 21 21 22 22 23 23 24 24
 25 25 26 26 27 27 28 28 29 29 30 30 31 31]
Test data accuarcy:  0.03225806451612903
